In the last lesson we encountered the Bitcoin protocol's [Version Handshake](https://en.bitcoin.it/wiki/Version_Handshake). We saw how Bitcoin network peers won't respond if you don't start the conversion with a `version` message.

But we cheated. I gave you a serialize `version` message and didn't tell you how I created it.

We were also lazy: we didn't parse the cryptic `payload` of the `version` message that our peer sent us.

We, too, we rude! After listening for our peer's `version` message we stopped listening and never received or responded to their `verack` message -- completing the handshake.

So you see, we have much to fix!

To begin, I'm going to redefine everything from last lesson. I'm going to rename `Message` -> `Packet`.

In the root of the `bitcoincorps` project directory you will now see a `ibd` folder, and inside it a `one.py ` file. `idb` stands for "initial block download" -- the goal of this project. After lesson I will add a new file to this directory containing all the important code we've written up through that lesson. This way we will have a nice library once this course is finished.

Check out `one.py` [here](./ibd/one.py)

And let's import all this code that we wrote last time:

In [21]:
from ibd.one import *

Let's also load the ["autoreload"](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=reload#module-IPython.extensions.autoreload) extension for Jupyter:

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Now we're back where we left off last time: we can send a hard-coded `version` message to a Bitcoin peer and make sense of the outermost attributes of the binary response we receive: "network magic", "command", "payload length", "payload checksum" and "payload" itself.

Next, we need to learn to 

In [7]:
import socket

PEER_IP = "35.187.200.6"
PEER_PORT = 8333

# magic "version" bytestring
VERSION = b'\xf9\xbe\xb4\xd9version\x00\x00\x00\x00\x00j\x00\x00\x00\x9b"\x8b\x9e\x7f\x11\x01\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x93AU[\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rV\xc5C\x9b:\xea\x89\x14/some-cool-software/\x01\x00\x00\x00\x01'

sock = socket.socket()
sock.connect((PEER_IP, PEER_PORT))

# initiate the "version handshake"
sock.send(VERSION)

# receive their "version" response
version_message = Packet.from_socket(sock)

print(version_message)

+----------+----------------------------------------------------------------------------------+
| Packet   |                                                                                  |
+==========+==================================================================================+
| command  | b'version'                                                                       |
+----------+----------------------------------------------------------------------------------+
| payload  | b"\x7f\x11\x01\x00\r\x04\x00\x00\x00\x00\x00\x00^\xb2^[\x00\x00\x00\x00\x0f\x04\ |
|          | x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\ |
|          | x00\x00\x00\x00\r\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0 |
|          | 0\x00\x00\x00\x00\x00\x00\x00\x00\x00K7\x0c\x0b\x9c\xd2\x1d<\x10/Satoshi:0.16.0/ |
|          | Y'\x08\x00\x01"                                                                  |
+----------+----------------------------

Same as last time, but you may notice that `Packet` instances look like pretty tables when printed. This is because I added a `Network.__str__` method to our `Packet` class definition, which is the function python calls when determining _how_ to print a given object. This `Packet.__str__` method simply runs a few of its values through this wonderful [tabulate](https://bitbucket.org/astanin/python-tabulate) program, which I added to our requirements.txt file.

Our next task is to parse this payload. Besides the "/Satoshi:0.16.0/" -- clearly a user agent -- the rest of the payload isn't human readable.

But have no fear -- we will decode the message payload in the same manner as we decoded the overall message structure in our `Packet.from_socket` method. 

[This chart](https://en.bitcoin.it/wiki/Protocol_documentation#version) from the protocol documentation will act as our blueprint.

![image](./images/version-message.png)

### Exercise #1: parse the version field contained within the payload of the version (a mouthful, I know!)

In [11]:
# binary_stream will look something like this:
# binary_stream = BytesIO(b'\x7f\x11\x01\x00\r\x04\x00\x00\x00\x00\x00\x00\xb4\x9dZ[\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\r\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00b\x8f\xc9N~]\x00\xb2\x10/Satoshi:0.16.0/p%\x08\x00\x01')

def read_version(binary_stream):
    ### your code here ###
    # step 1: read the correct number of bytes from `binary_stream` according to "field size" column above
    # step 2: interpret these bytes according to the "data type" column above
    bytes_ = binary_stream.read(4)
    int_ = bytes_to_int(bytes_)
    return int_

SyntaxError: invalid syntax (<ipython-input-11-4d218fce829f>, line 3)

In [9]:
import ipytest, pytest
import test_data

version_streams = test_data.make_version_streams()

def test_read_version_0():
    n = read_version(version_streams[0])
    assert n == 70015

def test_read_version_1():
    n = read_version(version_streams[1])
    assert n == 60001

def test_read_version_2():
    n = read_version(version_streams[2])
    assert n == 106
    
ipytest.run_tests(doctest=True)
ipytest.clean_tests("test_read_version*")

unittest.case.FunctionTestCase (test_read_version_0) ... ERROR
unittest.case.FunctionTestCase (test_read_version_1) ... ERROR
unittest.case.FunctionTestCase (test_read_version_2) ... ERROR
runTest (ipytest._DocTestCase) ... ok

ERROR: unittest.case.FunctionTestCase (test_read_version_0)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-9-732795f80e81>", line 9, in test_read_version_0
    n = read_version(version_streams[0])
NameError: name 'read_version' is not defined

ERROR: unittest.case.FunctionTestCase (test_read_version_1)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-9-732795f80e81>", line 13, in test_read_version_1
    n = read_version(version_streams[1])
NameError: name 'read_version' is not defined

ERROR: unittest.case.FunctionTestCase (test_read_version_2)
-----------------------------------------------------------


### Exercise #2: Given a version message binary stream, tell me whether the node that sent it can send a `pong` message ([hint](https://bitcoin.org/en/developer-reference#protocol-versions)).

In [6]:
def can_send_pong(binary_stream):
    ### your code here ###
    return read_version(binary_stream) >= 60001

In [7]:
version_streams = test_data.make_version_streams()

def test_can_send_pong_0():
    result = can_send_pong(version_streams[0])
    assert result == True

def test_can_send_pong_1():
    result = can_send_pong(version_streams[1])
    assert result == True

def test_can_send_pong_2():
    result = can_send_pong(version_streams[2])
    assert result == False
    
ipytest.run_tests(doctest=True)
ipytest.clean_tests("test_can_send_pong*")

unittest.case.FunctionTestCase (test_can_send_pong_0) ... ok
unittest.case.FunctionTestCase (test_can_send_pong_1) ... ok
unittest.case.FunctionTestCase (test_can_send_pong_2) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


Now we've got the hang of it.

Here's the outline of a `VersionMessage` class.

* It will have an `__init__` constructor method, which allows us to pass custom attrivutes like `timestamp` which may never take the same value twice if this code were running in production.
* It will have a hard-coded ("class variable")[https://www.toptal.com/python/python-class-attributes-an-overly-thorough-guide] of `command = b"version"`. With this decision we are setting a convention: any instance of this class or the other 26 XMessage classes we still have to implement will have a `.command` attribute so we can always know why kind of message we're dealing with.
* `VersionMessage.from_bytes` is also a convention that all 26 other `XMessage` classes will implement. Let's assume we are trying to handle an incoming `Packet` insance, which we will call `packet`. The purpose of this `VersionMessage.from_bytes` classmethod is to take the value of `packet.payload`, and based on `packet.command`, correctly read the payload values and instantiate an instance of the `XMessage` class. So, if `packet.command` is `b"block"` then we will need to read [these values](https://en.bitcoin.it/wiki/Protocol_documentation#block) and use them to instantiate a `BlockMessage` object. `packet.payload`s are raw bytes, which are much easier to read `n` bytes at a time if we first turn them into an `io.BytesIO` object -- so that's what we'll do.
* Lastly, you will notice `read_int`, `read_var_str`, `read_var_int` and `read_bool` methods, which we need to implement. Since we will doing some of these operations over and over again -- such as reading `n` bytes and interpreting them as a Python `int`, it makes sense to implement so-called "helper methods" to simplify our code, make it more testable and readable. 

In [8]:
class VersionMessage:

    command = b"version"

    def __init__(self, version, services, timestamp, addr_recv, addr_from, 
                 nonce, user_agent, start_height, relay):
        self.version = version
        self.services = services
        self.timestamp = timestamp
        self.addr_recv = addr_recv
        self.addr_from = addr_from
        self.nonce = nonce
        self.user_agent = user_agent
        self.start_height = start_height
        self.relay = relay

    @classmethod
    def from_bytes(cls, payload):
        stream = BytesIO(payload)
        
        version = read_int(stream, 4)
        services = read_int(stream, 8)
        timestamp = read_int(stream, 8)
        addr_recv = stream.read(26)
        addr_from = stream.read(26)
        nonce = read_int(stream, 8)
        user_agent = read_var_str(stream)
        start_height = read_int(stream, 4)
        relay = read_bool(stream)
        
        return cls(version, services, timestamp, addr_recv, addr_from, 
                   nonce, user_agent, start_height, relay)
    


![image](./images/version-message.png)

The `from_bytes` classmethod just translates the `Description` and `Data Type` columns of the above protocol documentation chart into python code. 

Here we encounter some "types" that we're familiar with abstracting the Bitcoin protocol's [general message structure](https://en.bitcoin.it/wiki/Protocol_documentation#Message_structure) with our `Packet` class -- `int32_t` / `uint64_t` / `int64_t` -- which are different types in a "low-level" language like C++, but are all equivalent to the `int` type in Python. Our previously implemented `bytes_to_int` can handle these just fine.

But we also encounter some new types: `net_addr`, `varstr`, and `bool`. Even worse, if we click on the [`varstr` link](https://en.bitcoin.it/wiki/Protocol_documentation#Variable_length_string) we see that it contains one additional type: `varint`. Worse still, the [`net_addr` link](https://en.bitcoin.it/wiki/Protocol_documentation#Network_address) contains a `time`, `services` and `IPv6/4` fields nominally of types `uint32`, `uint64_t` and `char[16]` but in order for us to make sense of what they hell them mean each requires parsing: the `time` integer as a Unix timestamp, the `services` integer as a damn "bitfield" (whatever that is!), and `IPv6/4` IP address as a 16 digit bytestring where the first 12 digits are always `00 00 00 00 00 00 00 00 00 00 FF FF` and only the last 4 matter! Oh, and remember how I mentioned that Satoshi usually, but not always, encoded his integers with "little endian" byte order (most significant digits is on the left)? Well the `port` attribute of `net_addr` is encoded "big endian", where the most significant digit is on the right. Yes, the exact opposite of everything else!!!

Hunker down for a looooooong lesson!

# "Integer" fields

In the last lesson we implemented `bytes_to_int(n)`. We'll start tackling all this mountain of work by implementing a small helper method `read_int(stream)` atop `bytes_to_int(n)` which first reads `n` from `stream` and then calls `bytes_to_int` with the bytes it read.

Adn we're going to create an argument`byte_order` which will default to `little` because almost every integer our program deals with will be little-endian encoded. But IP ports -- and soon other -- are big-endian encoded so we must allow callers to override this `bytes_order="little"` default value if they have a big-endian endcoded integer on their hands.

In [12]:
def read_int(stream, n, byte_order='little'):
    b = stream.read(n)
    return bytes_to_int(b, byte_order)

# "Boolean" fields

`bool` is the next simplest: it's a `1` or it's `0`. Actually, it's even simpler, huh? But we're going to resuse the code above so I'm introducing it second.

In fact, we could just use `read_int` and pass around `1`'s and `0`'s and our program would work just fine. After all, in Python the statement `1 == True and 0 == False` evaluates to `True` in Python. But Python gives us a built-in `bool` class for dealing with true-or-false, 1-or-0 values because it gives our programs greater clarity and readability.

### Exercise #3: implement `read_bool`

In [72]:
def read_bool(stream):
    bytes_ = stream.read(1)
#     if len(bytes_) != 1:
#         raise RuntimeError("Stream ran dry")
    integer =  bytes_to_int(bytes_)
    boolean = bool(integer)
    return boolean

In [18]:
import test_data

def test_read_bool_0():
    stream = test_data.make_stream(test_data.true_bytes)
    result = read_bool(stream)
    assert type(result) == bool
    assert result is True
    
def test_read_bool_1():
    stream = test_data.make_stream(test_data.false_bytes)
    result = read_bool(stream)
    assert type(result) == bool
    assert result is False
    
ipytest.run_tests(doctest=True)
ipytest.clean_tests("test_read_bool_*")

unittest.case.FunctionTestCase (test_read_bool_0) ... ok
unittest.case.FunctionTestCase (test_read_bool_1) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


Once you get your `read_bool` function to pass these tests by successfully reading `True` and `False` values, I want you to implement one more thing.

I want you to raise a RuntimeError if `stream.read(n)` doesn't return a byte string of length `n`. This is just a check to make sure that our program is running correctly.

In [19]:
def test_read_bool_2():
    stream = test_data.make_stream(b"")
    with pytest.raises(RuntimeError) as e_info:
        result = read_bool(stream)

ipytest.run_tests(doctest=True)
ipytest.clean_tests("test_read_bool_*")

unittest.case.FunctionTestCase (test_read_bool_2) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


# Timestamp fields

Network messages us Unix timestamps whenever they communicate a timestamp.

Here's how we interpret a Unix timestamp in Python

In [127]:
from datetime import datetime

def read_timestamp(stream):
    timestamp = read_int(stream, 8)
    return datetime.fromtimestamp(timestamp)


# "Variable Length" fields

Next comes `var_str`, the type of the "User Agent", which is basically an advertisement of the Bitcoin software implementation that the node is using. You can see a listing of popular values [here](https://bitnodes.earn.com/nodes/).

["Variable Length Strings"](https://en.bitcoin.it/wiki/Protocol_documentation#Variable_length_string) are used for values that string fields undependable length. This technique strives to use only the space it needs. It does this by prepending a "variable length integer" in front of the string value being communicated. This `var_int` tells the recipient how many bytes they need to read to get the string value being communicated. This is kind of similar to hopw the payload bytes are handled in our `Packet.from_bytes` -- first we read `length` and then we read `lenght`-many bytes to get our raw payload. Same idea here, but now the length of the string isn't an integer, but a "variable length integer".

How does `var_int` work?

The first byte of a `var_int` is a marker which says how many bytes come after it:
* `0xFF`: 8 byte integer follows
* `0xFE`: 4 byte integer follows
* `0xFD`: 2 byte integer follows
* < `0xFD`: interpret first byte as a 1 byte integer

### Exercise #4:  implement `var_int`, since `var_str` depends on it.

In [137]:
def read_var_int(stream):
    i = read_int(stream, 1)
    if i == 0xff:
        return bytes_to_int(stream.read(8))
    elif i == 0xfe:
        return bytes_to_int(stream.read(4))
    elif i == 0xfd:
        return bytes_to_int(stream.read(2))
    else:
        return i

In [99]:
import ipytest, pytest
import test_data as td

enumerated = (
    (td.eight_byte_int, td.eight_byte_var_int),
    (td.four_byte_int, td.four_byte_var_int),
    (td.two_byte_int, td.two_byte_var_int),
    (td.one_byte_int, td.one_byte_var_int),
)

def test_read_var_int():
    for correct_int, var_int in enumerated:
        stream = td.make_stream(var_int)
        calculated_int = read_var_int(stream)
        assert correct_int == calculated_int

ipytest.run_tests(doctest=True)
ipytest.clean_tests("test_read_var_int*")

unittest.case.FunctionTestCase (test_read_var_int) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... ok
runTest (ipytest._DocTestCase) ... 

var int first bytes 255
var int first bytes 254
var int first bytes 253
var int first bytes 7


ok

----------------------------------------------------------------------
Ran 15 tests in 0.036s

OK


Now that we have that out of the way:

### Exercise #5: Implement `read_var_str`

In [138]:
def read_var_str(stream):
    length = read_var_int(stream)
    string = stream.read(length)
    return string

In [46]:
import ipytest, pytest
import test_data as td

enumerated = (
    (td.short_str, td.short_var_str),
    (td.long_str, td.long_var_str),
)

def test_read_var_str():
    for correct_byte_str, var_str in enumerated:
        stream = td.make_stream(var_str)
        calculated_byte_str = read_var_str(stream)
        assert correct_byte_str == calculated_byte_str

ipytest.run_tests(doctest=True)
ipytest.clean_tests("test_read_var_str*")

unittest.case.FunctionTestCase (test_read_var_str) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


# `services` field

[The version section of the protocol docs](https://en.bitcoin.it/wiki/Protocol_documentation#version) provides us with the following guide for interpreting the `services` field of the `version` payload:

![image](images/services.png)

The type of this field is "bitfield". [Check out the wikipedia entry](https://en.wikipedia.org/wiki/Bit_field) for a better explanation that I can provide.

In a bitfield every bit holds some pre-defined meaning. This is a 8 byte / 64 bit bitfield (remember, a byte is just a collection of 8 bits so 8 bytes is 8*8=64 bits).


From the table above we can see that the least significant digit in the binary representation (decimal value `2^0=1`) represents `NODE_NETWORK`, or whether this peer "can be asked for full blocks or just headers".

The second least-significant digit (decimal value `2^1=2`) NODE_GETUTXO

The third least-significant digit (decimal value `2^2=4`)NODE_BLOOM

The fourth least-significant digit (decimal value `2^3=8`) NODE_WITNESS

The eleventh least-significant digit (decimal value `2^10=1024`) NODE_NETWORK_LIMITED

The rest of the bits (decimal values `2*n` where n in {4, 5, 6, 7, 8, 9, 11, 12, ..., 63} have no meaning, yet.

So, in order to interpret this field we need to look up the nth bit in the table above and see if it means anything.

So, in python we want to be able to basically produce a dictionary like this for every node we connect to. This would allow us to look up what services that node offers _by name_.

```
{
    'NODE_NETWORK': True,
    'NODE_GETUTXO': False,
    'NODE_BLOOM': True,
    'NODE_WITNESS': False,
    'NODE_NETWORK_LIMITED': True,
}
```

Furthermore, we could write a function that produces this lookup table for us given an integer bitfield and a magical `check_bit(n)` function:

```
def read_services(stream):
    n = read_int(stream, 4)
    return {
        'NODE_NETWORK': check_bit(services_int, 0),           # 1    = 2**0
        'NODE_GETUTXO': check_bit(services_int, 1),           # 2    = 2**1
        'NODE_BLOOM': check_bit(services_int, 2),             # 4    = 2**2
        'NODE_WITNESS': check_bit(services_int, 3),           # 8    = 2**3
        'NODE_NETWORK_LIMITED': check_bit(services_int, 10),  # 1024 = 2**10
    }
```

For now, I'm just going to give you a definition of the magical `check_bit` function:

In [89]:
def check_bit(number, index):
    """See if the bit at `index` in binary representation of `number` is on"""
    mask = 1 << index
    return bool(number & mask)

def services_int_to_dict(services_int):
    return {
        'NODE_NETWORK': check_bit(services_int, 0),           # 1    = 2**0
        'NODE_GETUTXO': check_bit(services_int, 1),           # 2    = 2**1
        'NODE_BLOOM': check_bit(services_int, 2),             # 4    = 2**2
        'NODE_WITNESS': check_bit(services_int, 3),           # 8    = 2**3
        'NODE_NETWORK_LIMITED': check_bit(services_int, 10),  # 1024 = 2**10
    }

def read_services(stream):
    services_int = read_int(stream, 8)
    return services_int_to_dict(services_int)

Here's some `read_services` outputs for some possible inputs:

In [41]:
from pprint import pprint

bitfields = [
    1,
    8,
    1 + 8,
    1024,
    8 + 1024,
    1 + 2 + 4 + 8 + 1024,
    2**5 + 2**9 + 2**25,
]

for bitfield in bitfields:
    pprint(f"(n={bitfield})")
    stream = BytesIO(int_to_bytes(bitfield, 4))
    pprint(read_services(stream))
    print()

'(n=1)'
{'NODE_BLOOM': False,
 'NODE_GETUTXO': False,
 'NODE_NETWORK': True,
 'NODE_NETWORK_LIMITED': False,
 'NODE_WITNESS': False}

'(n=8)'
{'NODE_BLOOM': False,
 'NODE_GETUTXO': False,
 'NODE_NETWORK': False,
 'NODE_NETWORK_LIMITED': False,
 'NODE_WITNESS': True}

'(n=9)'
{'NODE_BLOOM': False,
 'NODE_GETUTXO': False,
 'NODE_NETWORK': True,
 'NODE_NETWORK_LIMITED': False,
 'NODE_WITNESS': True}

'(n=1024)'
{'NODE_BLOOM': False,
 'NODE_GETUTXO': False,
 'NODE_NETWORK': False,
 'NODE_NETWORK_LIMITED': True,
 'NODE_WITNESS': False}

'(n=1032)'
{'NODE_BLOOM': False,
 'NODE_GETUTXO': False,
 'NODE_NETWORK': False,
 'NODE_NETWORK_LIMITED': True,
 'NODE_WITNESS': True}

'(n=1039)'
{'NODE_BLOOM': True,
 'NODE_GETUTXO': True,
 'NODE_NETWORK': True,
 'NODE_NETWORK_LIMITED': True,
 'NODE_WITNESS': True}

'(n=33554976)'
{'NODE_BLOOM': False,
 'NODE_GETUTXO': False,
 'NODE_NETWORK': False,
 'NODE_NETWORK_LIMITED': False,
 'NODE_WITNESS': False}



Hopefully that kind of makes sense.

### Exercise #6: complete these useless functions to hammer home you understanding of this strange `services` "bitfield"

In [42]:
def offers_node_network_service(services_bitfield):
    # given integer services_bitfield, return whether the NODE_NETWORK bit is on
    return services_int_to_dict(services_bitfield)['NODE_NETWORK']

def offers_node_bloom_and_node_witness_services(services_bitfield):
    # given integer services_bitfield, return whether the 
    # NODE_BLOOM and NODE_WITNESS bits are on
    return services_int_to_dict(services_bitfield)['NODE_BLOOM'] \
        and services_int_to_dict(services_bitfield)['NODE_WITNESS']

In [43]:
import ipytest, pytest

def test_services_0():
    assert offers_node_network_service(1) is True
    assert offers_node_network_service(1 + 8) is True
    assert offers_node_network_service(4) is False
    

def test_services_1():
    assert offers_node_bloom_and_node_witness_services(1) is False
    assert offers_node_bloom_and_node_witness_services(1 + 8) is False
    assert offers_node_bloom_and_node_witness_services(4 + 8) is True
    
ipytest.run_tests(doctest=True)
ipytest.clean_tests("test_services*")

unittest.case.FunctionTestCase (test_services_0) ... ok
unittest.case.FunctionTestCase (test_services_1) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


In [299]:
def services_offered(bitfield):
    return [k for k, v in services_int_to_dict(bitfield).items() if v]

In [44]:
import ipytest, pytest

raw_version_to_services_offered = {
    b'\x7f\x11\x01\x00\r\x04\x00\x00\x00\x00\x00\x00\xa3\xfcY[\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffh\x05=\x04\x9b\xf8\r\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x88\x88\x18\xfaf\x93\x92\x97\x10/Satoshi:0.16.0/\x0eq\x07\x00\x01': ['NODE_NETWORK', 'NODE_BLOOM', 'NODE_WITNESS', 'NODE_NETWORK_LIMITED'],
    b'r\x11\x01\x00\x01\x00\x00\x00\x00\x00\x00\x00\xf8\xfcY[\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffh\x05=\x04\xc3N\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffR\x16x\xe8 \x8d+L\xd2\xab\xc1\xd8\x0c\xeb\x0f/Satoshi:0.9.3/n\x80\x06\x00\x01': ['NODE_NETWORK'],
    b'\x7f\x11\x01\x00\r\x00\x00\x00\x00\x00\x00\x00|\xfcY[\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffh\x05=\x04\xc9.\r\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8b3;\xd3\xde-Y\x99\x10/Satoshi:0.15.0/&%\x08\x00\x01': ['NODE_NETWORK', 'NODE_BLOOM', 'NODE_WITNESS'],
    b'\x7f\x11\x01\x00\x0c\x04\x00\x00\x00\x00\x00\x00y\xfcY[\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffh\x05=\x04\x8e\xa4\x0c\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00`4"\xe2B\xc6\x82\r\x10/Satoshi:0.16.0/&%\x08\x00\x01': ['NODE_BLOOM', 'NODE_WITNESS', 'NODE_NETWORK_LIMITED'],
}

def test_services_offered():
    for raw_version, services_offered in raw_version_to_services_offered.items():
        v = VersionMessage.from_bytes(raw_version)
        calculated_services_offered = [k for k, v in services_int_to_dict(v.services).items() if v]
        assert set(services_offered) == set(calculated_services_offered)

ipytest.run_tests(doctest=True)
ipytest.clean_tests("test_services_offered*")

unittest.case.FunctionTestCase (test_services_offered) ... ERROR

ERROR: unittest.case.FunctionTestCase (test_services_offered)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-44-2d1def9011d5>", line 12, in test_services_offered
    v = VersionMessage.from_bytes(raw_version)
  File "<ipython-input-9-1a5fbd798093>", line 28, in from_bytes
    user_agent = read_var_str(stream)
NameError: name 'read_var_str' is not defined

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


# "Network Address" Type

[`net_addr`](https://en.bitcoin.it/wiki/Protocol_documentation#Network_address) is the most complicated, so we'll handle it last.

Network addresses require we interpret 4 new kinds of data:

1. `time`: Unix timestamp. Already done.
2. `services`: integer bitfield. Already done.
3. `IP address`: complicated ...
4. `port`: big-endian encoded `int`

Here's a Python class abstracting this "Network Address" type. Just two methods remain to be implemented: `read_ip` and `read_port`:

In [ ]:
class Address:

    def __init__(self, services, ip, port, time):
        self.services = services
        self.ip = ip
        self.port = port
        self.time = time

    @classmethod
    def from_bytes(cls, bytes_, version_msg=False):
        stream = BytesIO(bytes_)
        return cls.from_stream(stream, version_msg)
    
    @classmethod
    def from_stream(cls, stream, version_msg=False):
        if version_msg:
            time = None
        else:
            time = read_timestamp(stream)
        services = read_services(stream)
        ip = stream.read(16)
        port = read_int(stream, 2, byte_order='big')
        return cls(services, ip, port, time)
    
    def __repr__(self):
        return f"<Address {self.ip}:{self.port}>"

## "Network Addess > IPv6/4" field

Since these "network addresses" are sort of their own "data type", let's make a class to abstract them.

Here's my best translation of the docs:

![image](images/network-address.png)

In [96]:
def read_ip(stream):
    bytes_ = stream.read(16)
    return ip_address(bytes_)

## "Network Address > Port" Field

This is just 2 byte integer -- but it's encoded with the opposite byte order of what we usually read using `read_bytes`. But have no fear, `read_bytes` takes an optional `byte_order` parameter which defaults to `little` -- since we're usually reading little-endian encoded messages. But if we set it to `big`, then `read_int` will successfully read the "big endian" / "network byte order" port number.

In order to have clean, testable code we will define another helper method: `read_port`

In [140]:
def read_port(stream):
    return read_int(stream, 2, byte_order="big")

# Parsing a complete Version response

Let's put together all the little helper function and helper classes we've so dilligently written and parse the payload of the version_message we downloaded at the beginning of this lesson.

In [19]:
import socket
from ibd.two import VersionMessage # get the final version ...

PEER_IP = "35.187.200.6"

PEER_PORT = 8333

# magic "version" bytestring
VERSION = b'\xf9\xbe\xb4\xd9version\x00\x00\x00\x00\x00j\x00\x00\x00\x9b"\x8b\x9e\x7f\x11\x01\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x93AU[\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rV\xc5C\x9b:\xea\x89\x14/some-cool-software/\x01\x00\x00\x00\x01'

sock = socket.socket()
sock.connect((PEER_IP, PEER_PORT))

# initiate the "version handshake"
sock.send(VERSION)

# receive their "version" response
pkt = Packet.from_socket(sock)

msg = VersionMessage.from_bytes(pkt.payload)
print(msg)

+------------------+----------------------------------------------------------------------------------+
| VersionMessage   |                                                                                  |
+==================+==================================================================================+
| version          | 70015                                                                            |
+------------------+----------------------------------------------------------------------------------+
| services         | {'NODE_NETWORK': True, 'NODE_GETUTXO': False, 'NODE_BLOOM': True, 'NODE_WITNESS' |
|                  | : True, 'NODE_NETWORK_LIMITED': True}                                            |
+------------------+----------------------------------------------------------------------------------+
| timestamp        | 2018-07-30 02:30:36                                                              |
+------------------+--------------------------------------------

Boom! This is basically the same code we finished the last lesson with, but our magical `version_report` function and all the functions it calls are able to decipher what this cryptic message _means_!